<a href="https://colab.research.google.com/github/PradNir/Gen-AI-Driven-Cover-Letter-and-Cold-Email-Generator-for-Job-Applications/blob/main/EmotionDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, LSTM, Dense
from sklearn.model_selection import train_test_split
import networkx as nx
import cv2
import os

In [ ]:
# Load text data
text_data = pd.read_csv('/content/drive/MyDrive/EmotionDetection/iemocap_full_dataset.csv')

In [ ]:
# Function to preprocess image
def preprocess_image(image):
    # Resize image to a fixed size (e.g., 48x48 pixels)
    resized_image = cv2.resize(image, (48, 48))
    # Convert image to grayscale
    grayscale_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
    # Normalize pixel values to [0, 1]
    normalized_image = grayscale_image / 255.0
    return normalized_image

In [ ]:
# Function to extract label from filename
def extract_label(filename):
    # Extract emotion label from filename
    label = int(filename.split('.')[0][-2:])
    return label


In [ ]:
# Load labels
labels = []
for filename in os.listdir('/content/drive/MyDrive/EmotionDetection/CK+'):
    if filename.endswith('.png'):
        label = extract_label(filename)
        labels.append(label)
labels = np.array(labels)

In [ ]:
# Path to the directory containing CK+ images
ck_plus_images_dir = '/content/drive/MyDrive/EmotionDetection/CK+'

In [ ]:
# Initialize lists to store images and labels
images = []
labels = []

# Define a mapping of emotion labels to numerical values
emotion_label_map = {
    'Anger': 0,
    'Contempt': 1,
    'Disgust': 2,
    'Fear': 3,
    'Happy': 4,
    'Sadness': 5,
    'Surprise': 6
}


# Loop through each directory (corresponding to each emotion label)
for emotion_dir in os.listdir(ck_plus_images_dir):
    # Skip non-directory files
    if not os.path.isdir(os.path.join(ck_plus_images_dir, emotion_dir)):
        continue

    # Map the emotion label to its numerical value
    label = emotion_label_map.get(emotion_dir)
    if label is None:
        continue  # Skip if the emotion label is not defined in the mapping

    # Loop through each image file in the directory
    for filename in os.listdir(os.path.join(ck_plus_images_dir, emotion_dir)):
        if filename.endswith('.png'):  # Assuming images are in PNG format
            # Read the image
            image = cv2.imread(os.path.join(ck_plus_images_dir, emotion_dir, filename))
            if image is None:
                continue  # Skip if image loading fails
            # Preprocess the image
            preprocessed_image = preprocess_image(image)
            # Add the preprocessed image to the list of images
            images.append(preprocessed_image)
            # Add the label to the list of labels
            labels.append(label)

# Convert lists to NumPy arrays
images = np.array(images)
labels = np.array(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
# Initialize and train a Support Vector Machine (SVM) classifier
svm_classifier = SVC(kernel='linear')
svm_classifier.fit(X_train.reshape(len(X_train), -1), y_train)

In [ ]:
# Initialize and train a Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=100)
rf_classifier.fit(X_train.reshape(len(X_train), -1), y_train)

In [ ]:
# Apply K-Means clustering
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(X_train.reshape(len(X_train), -1))

In [ ]:
# Define the CNN model for facial expression analysis
def build_cnn_model():
    model = tf.keras.Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(7, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# Build the CNN model
cnn_model = build_cnn_model()
cnn_model.fit(X_train.reshape(-1, 48, 48, 1), y_train, epochs=10)

In [ ]:
# Evaluate SVM model
svm_accuracy = svm_classifier.score(X_test.reshape(len(X_test), -1), y_test)

# Evaluate Random Forest model
rf_accuracy = rf_classifier.score(X_test.reshape(len(X_test), -1), y_test)

print("SVM Accuracy:", svm_accuracy)
print("Random Forest Accuracy:", rf_accuracy)


In [ ]:
# Evaluate CNN model
cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test.reshape(-1, 48, 48, 1), y_test)

print("CNN Accuracy:", cnn_accuracy)


In [ ]:
# Save the pre-trained CNN model
cnn_model.save('emotion_recognition_model.h5')


In [ ]:
import cv2

# Load the pretrained CNN model for emotion detection
cnn_model = tf.keras.models.load_model('/content/emotion_recognition_model.h5')

# Define the emotion labels
emotion_labels = {0: 'Anger', 1: 'Contempt', 2: 'Disgust', 3: 'Fear', 4: 'Happy', 5: 'Sadness', 6: 'Surprise'}

# Function to detect emotion from an image frame
def detect_emotion(frame):
    # Preprocess the frame
    resized_frame = cv2.resize(frame, (48, 48))
    grayscale_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
    normalized_frame = grayscale_frame / 255.0
    input_frame = np.expand_dims(normalized_frame, axis=-1)
    input_frame = np.expand_dims(input_frame, axis=0)  # Add batch dimension

    # Perform emotion prediction
    predictions = cnn_model.predict(input_frame)
    predicted_class = np.argmax(predictions)
    emotion_label = emotion_labels[predicted_class]

    return emotion_label

# Function to perform real-time emotion detection from the camera
def real_time_emotion_detection():
    # Initialize the camera
    camera = cv2.VideoCapture(0)

    while True:
        # Capture frame-by-frame
        ret, frame = camera.read()
        if not ret:
            break

        # Detect emotion in the frame
        emotion_label = detect_emotion(frame)

        # Display the emotion label on the frame
        cv2.putText(frame, emotion_label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Real-time Emotion Detection', frame)

        # Break the loop when 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close all OpenCV windows
    camera.release()
    cv2.destroyAllWindows()

# Perform real-time emotion detection
real_time_emotion_detection()


In [ ]:
print(cnn_model.summary())


In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
# Initialize camera
camera = cv2.VideoCapture(0)

In [ ]:
while True:
    # Capture frame-by-frame
    ret, frame = camera.read()

    # Check if frame capture was successful
    if not ret:
        print("Error: Failed to capture frame from the camera.")
        break
# Display the frame
    cv2_imshow(frame)

    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close OpenCV windows
camera.release()
cv2.destroyAllWindows()